In [3]:
!nvidia-smi

Mon Jun 26 08:52:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [5]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.7 MB/s eta 0:00:00
Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


In [1]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [6]:
!wget https://www.github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2023-06-26 09:05:31--  https://www.github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving www.github.com (www.github.com)... 192.30.255.112
Connecting to www.github.com (www.github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip [following]
--2023-06-26 09:05:31--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-06-26 09:05:32--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.1

In [7]:
data = load_from_disk('samsum_dataset')
data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
def convert_examples_to_features(batch):
  input_encodings = tokenizer(batch['dialogue'], max_length = 1024, truncation = True)
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(batch['summary'], max_length = 128, truncation = True)
  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask' : input_encodings['attention_mask'],
      'labels' : target_encodings['input_ids']
  }

In [13]:
data_samsum_pt = data.map(convert_examples_to_features, batched = True)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [16]:
#Training
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = 'pegasus-sumsum', num_train_epochs = 1, warmup_steps = 500,
    per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
    weight_decay = 0.01, logging_steps = 10, evaluation_strategy = 'steps',
    eval_steps = 500, save_steps = 1e6,
    gradient_accumulation_steps = 16
)

In [20]:
trainer = Trainer(model = model_pegasus, args = training_args,
                  tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = data_samsum_pt['test'],
                  eval_dataset = data_samsum_pt['validation']
                  )

In [21]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0461518156762217, metrics={'train_runtime': 190.6381, 'train_samples_per_second': 4.296, 'train_steps_per_second': 0.268, 'total_flos': 312908679512064.0, 'train_loss': 3.0461518156762217, 'epoch': 1.0})

In [33]:
#Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
  '''
  Split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list of elements
  '''
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_data(dataset, metric, model, tokenizer,
                                  batch_size = 16, device = device,
                                  column_text = 'article',
                                  column_summary = 'highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batches, target_batch in tqdm(
      zip(article_batches, target_batches), total = len(article_batches)):
      inputs = tokenizer(article_batches, max_length = 1024, truncation = True,
                         padding = 'max_length', return_tensors = "pt")
      summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                                 attention_mask = inputs['attention_mask'].to(device),
                                 length_penalty = 0.8, num_beams = 8, max_length = 128)
      decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True,
                                            clean_up_tokenization_spaces = True)
                          for s in summaries]
      decoded_summaries = [d.replace(""," ") for d in decoded_summaries]

      metric.add_batch(predictions = decoded_summaries, references = target_batch)
  score = metric.compute()
  return score

In [34]:
rouge_names = ['rouge1','rouge2','rougeL','rougeLsum']
rouge_metric = load_metric('rouge')

In [35]:
score = calculate_metric_on_test_data(
    data['test'][:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary = 'summary'
)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])

100%|██████████| 5/5 [00:16<00:00,  3.23s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.022373,0.0,0.02209,0.02219


In [36]:
model_pegasus.save_pretrained('pegasus-samsun-model')

In [37]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')